主要参考：

+ [1] https://github.com/Ashish-Gupta03/Hierarchical-Attention-Networks-for-Document-Classification-NAACL-2016
+ [2] https://github.com/charlesdong1991/interpretable-han-for-document-classification-with-keras

实现[1]  模型代码在本目录下的model.py中.仔细看了之后发现也是最初那个实现的复现，心疼。。。

# 数据处理

In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,ModelCheckpoint

Using TensorFlow backend.


In [2]:
max_sen_len = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1
EMBEDDING_FILE = 'E:/jupter/TheSecond-Paper/word_embedding/en_model.txt'

## 读取数据集

In [3]:
def clean_str(string):
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [11]:
train_df =pd.read_csv('../labeledTrainData.tsv',sep='\t')

In [12]:
texts, labels = [], []
for i in range(train_df['review'].shape[0]):
    text = train_df['review'][i]
    texts.append(clean_str(text))
    labels.append(train_df['sentiment'][i])

In [14]:
type(texts)

list

In [7]:
labels = np.array(labels)

In [15]:
tok = Tokenizer(num_words=MAX_NUM_WORDS)
tok.fit_on_texts(texts) 
#tok.word_index 更新内部词典

seq = tok.texts_to_sequences(texts) #Transforms each text in texts to a sequence of integers.

data = pad_sequences(seq,maxlen=max_sen_len)

print(data.shape)

(25000, 100)


In [16]:
data[:2]

array([[  157,    60,    10,   102,     6,    88,    80,    44,    21,
           91,   786,   240,     9,   123,   351,     2,   198,   121,
            3,  7681,   746,     2,  3622,  2053,     8,    11,    17,
            6,     3,   243,   488,  1877,     6,   367,    28,     4,
            1,    88,  1019,    80,   122,     5,  1686,    11,  1206,
           18,     6,    26,  2517,    69,    16,    30,     1,   690,
          201,   521,    11,   873,  7261,    69,    10,    89,   120,
           85,    80,    68,    27,   272,   496,  4584,  3552,    10,
          120,    11,    15,     3,   189,    26,     6,   343,    33,
          576,   324,    18,   375,   227,    40,    28,     4,     1,
           88, 19375, 19376,    10,   440,    26,     6,    21,     1,
         1564],
       [  171,   287,   961,  1330,     1,  3676,   308,    16,   815,
         3787,    12,    67,    62,     1,  3648,  4066,     5,     1,
          271,   540,   312,   269,    15,   272,   179,     

In [17]:
data = np.expand_dims(data, axis=1)
# Insert a new axis that will appear at the `axis` position in the expanded array shape.把新加的轴放在axis=1的位置

In [18]:
data.shape

(25000, 1, 100)

In [26]:
idx = tok.word_index
labels_cat = np.expand_dims(labels,axis=1)

In [27]:
labels_cat.shape

(25000, 1)

In [29]:
np.array(labels).ndim

1

labels本来只有一维，axis=0,然后在1的位置扩展了

针对np.expand_dims:

```note:: Previous to NumPy 1.13.0, neither ``axis < -a.ndim - 1`` nor``axis > a.ndim`` raised errors or put the new axis where documented.
Those axis values are now deprecated and will raise an AxisError in the future.```

在np1.13.0版本以前，当axis参数< 或者 axis>  的时候会引起错误，或者将axis放在记录的位置，以后这样会报错。

In [30]:
labels_cat.shape,np.__version__

((25000, 1), '1.16.2')

In [31]:
x_train,x_val,y_train,y_val = train_test_split(data,labels_cat,test_size=0.12)

## 读取预训练词向量

In [32]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

## 构建词向量矩阵

In [33]:
num_words = min(MAX_NUM_WORDS, len(idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in idx.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# 模型构建

In [42]:
from model import *
from importlib import reload
# reload(model)
model = createHierarchicalAttentionModel(max_sen_len, embWeights=embedding_matrix,
                                                        embeddingSize = 300, vocabSize = min(MAX_NUM_WORDS, len(idx) + 1))

E:\jupter\Blog_Study\HATT实现\HATT实现4\model.py:154: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  model = Model(input=docInp,output=docOut)


In [43]:
filepath="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
callbacks_list = [checkpoint, early]

In [45]:
model.fit(x_train, y_train, batch_size=64, epochs=4,validation_data=(x_val, y_val),callbacks = callbacks_list,verbose=1)

Train on 22000 samples, validate on 3000 samples
Epoch 1/4
22000/22000 [==============================] - 169s 8ms/step - loss: 0.4315 - acc: 0.8044 - val_loss: 0.3666 - val_acc: 0.8320

Epoch 00001: val_acc improved from 0.77633 to 0.83200, saving model to weights_base.best.hdf5
Epoch 2/4
22000/22000 [==============================] - 167s 8ms/step - loss: 0.3781 - acc: 0.8334 - val_loss: 0.3685 - val_acc: 0.8350

Epoch 00002: val_acc improved from 0.83200 to 0.83500, saving model to weights_base.best.hdf5
Epoch 3/4
22000/22000 [==============================] - 168s 8ms/step - loss: 0.3678 - acc: 0.8387 - val_loss: 0.3436 - val_acc: 0.8507

Epoch 00003: val_acc improved from 0.83500 to 0.85067, saving model to weights_base.best.hdf5
Epoch 4/4
22000/22000 [==============================] - 171s 8ms/step - loss: 0.3461 - acc: 0.8501 - val_loss: 0.3310 - val_acc: 0.8640

Epoch 00004: val_acc improved from 0.85067 to 0.86400, saving model to weights_base.best.hdf5


唯一的差别就是词向量不一样，参考的代码得到的准确率：

 loss: 0.0627 - acc: 0.9796 - val_loss: 0.2265 - val_acc: 0.9227
 
 相差12%左右，差距很大，所以词向量很重要。实现了就好，谢谢大佬。
 
 这个用TensorFlow跑的和用Theano跑的，速度上差了很多

In [47]:
loss,score=model.evaluate(x_val,y_val)
print('score ',score)
print('loss ',loss)
model.load_weights(filepath)

3000/3000 [==============================] - 10s 3ms/step
score  0.8640000001589457
loss  0.3310120234489441


In [49]:
test_df = pd.read_csv('../testData.tsv', sep='\t')

In [50]:
texts_test = [] 
for i in range(test_df['review'].shape[0]):
    text = test_df['review'][i]
    texts_test.append(clean_str(text))

In [51]:
test_seq = tok.texts_to_sequences(texts_test)
test_data = pad_sequences(test_seq,maxlen=max_sen_len)
test_data = np.expand_dims(test_data, axis=1)

In [53]:
pred = model.predict(test_data,batch_size=1024)

In [54]:
pred[:3]

array([[0.9816089 ],
       [0.04317772],
       [0.3155845 ]], dtype=float32)

In [55]:
final_pred = []
for i in pred:    
    if i>0.9:
        final_pred.append(1)
    else: final_pred.append(0)   #其实不是很理解为什么>0.9的就是积极，剩下的就是消极。。。

In [56]:
subm = pd.DataFrame(data=[],columns=['id','sentiment'])#read_csv('sampleSubmission.csv')
subm['sentiment'] = final_pred
subm['id'] = test_df['id']

In [57]:
subm.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0
